# Paquetes

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import tldextract
import jellyfish
from joblib import dump, load

# Importar Dataset

Luego de haber tuneado y evaluado los modelos con el Train set, los elegimos según su performance en el Test set. Elegido el mejor de los modelos con el set de Test, entrenamos dicho modelo elegido con TODOS los datos. Acá importamos el dataset completo.

In [2]:
# Importar dataset completo

# Entrenar Modelo

En esta parte entrenamos al modelo elegido y guardamos el objeto en un archivo **joblib**, para luego ser importado por el script de la aplicación y poder ser utilizado de manera directa en las predicciones.

In [12]:
# Entrenamos el modelo con el dataset completo.

In [11]:
# Exportamos el archivo con el modelo entrenado.
dump(model, 'trained_model.joblib') 

In [5]:
rf = load('/home/jcc/JuanCruz_Ubuntu/Repositorios_Activos/Proyecto_CDD/Proyecto A/00 - Creación Dataset y Variables/trained_rf.joblib')

In [9]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# Prueba Aplicación

Podemos importar el módulo creado para usar la aplicación en un set de URLs que querramos.

In [2]:
from Aplicacion import DetectarPhishing

In [3]:
detector = DetectarPhishing()

Instancia del detector fue creada


In [5]:
data_urls['suffix'].mask(data_urls['suffix'] != '', 1)

NameError: name 'data_urls' is not defined

In [6]:
pd.get_dummies(prueba['suffix']=='').columns[0] == False

NameError: name 'prueba' is not defined

In [4]:
metric_domains = [
    'amazon', 
    'instagram', 
    'google', 
    'whatsapp',  
    'twitter',
    'facebook',
    'yahoo', 
    'wikipedia',
    'baidu',
    'paypal', 
    'mail', 
    'sfexpress' ,
    'onedrive',
    'excel', 
    'square', 
    'mail', 
    'office365', 
    'irs', 
    'tencent', 
    'creditagrecole s.a.',
    'microsoft',
    'blogspot',
    'onedrive',
    'payment',
    'hsbc',
    'secure',
    'help',
    'banco',
    'bank',
    'support',
    'rakuten',
    'steam',
    'olx'
]

In [ ]:
urls = ['https://git-scm.com/docs/gitignore', 
        'https://scikit-learn.org/stable/modules/model_persistence.html',
        'https://www.lanacion.com.ar/',
        'https://articulo.mercadolibre.com.ar/MLA-737102690-sommier-king-la-cardeuse-native-400-200x200-_JM?variation=95000501490#reco_item_pos=2&reco_backend=promotions-sorted-by-score-mla-A&reco_backend_type=low_level&reco_client=seller-promotions&reco_id=574ab7bd-21ab-43a8-a868-69b5785b0fc9&deal_print_id=25c97840-04f2-11ec-85d9-336a60e0da68&c_id=carouseldynamic-home&c_element_order=undefined&c_campaign=OFERTAS-X-24-HORAS&c_uid=25c97840-04f2-11ec-85d9-336a60e0da68',
        'https://www.clarin.com/', 
        'https://amoueaom-cc-jp.wnejpkv.cn/',
        'http://abdcenter.rhinosme-stagging.com/.kkklll', 
        ' https://foundations-admin-10000000003216566325623260.ml/checkpoint_next.php ']

prueba = pd.DataFrame({'url':urls})

In [22]:
        data_urls = prueba
        # Descomposición del URL.

        data_urls['scheme'] = data_urls['url'].apply(lambda x: urlparse(x).scheme)

        data_urls['domain_complete'] = data_urls['url'].apply(lambda x: urlparse(x).netloc)
        data_urls['domain_complete'] = data_urls['domain_complete'].str.replace('www.', '')
        data_urls['domain_complete'] = data_urls['domain_complete'].str.replace('www', '')

        data_urls['domain'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).domain)
        data_urls['subdomain'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).subdomain)
        data_urls['suffix'] = data_urls['domain_complete'].apply(lambda x: tldextract.extract(x).suffix)

        data_urls['subdomain'] = data_urls['subdomain'] + '.'
        data_urls['subdomain'] = data_urls['subdomain'].replace('.', '')

        data_urls['domain_subdomain'] = data_urls['subdomain'] + data_urls['domain']

        data_urls['path'] = data_urls['url'].apply(lambda x: urlparse(x).path)

        # Creación de Variables.

        # Variables del dominio
        # Cuenta los puntos
        data_urls['dom_n_puntos'] = data_urls['domain_subdomain'].str.count('\\.')
        data_urls['dom_n_guion'] = data_urls['domain_subdomain'].str.count('\\-')
        data_urls['dom_n_guionbajo'] = data_urls['domain_subdomain'].str.count('\\_')

        # Cuenta el largo total del dominio + subdominio
        data_urls['dom_len_tot'] = data_urls['domain_subdomain'].str.len()
        # Cuenta el largo del dominio y subdominio por separado
        data_urls['dom_len'] = data_urls['domain'].str.len()
        data_urls['dom_len_sub'] = data_urls['subdomain'].str.len()
        data_urls['url_len'] = data_urls['url'].str.len()
        # Cuenta vocales
        data_urls['dom_vocales'] = data_urls['domain_subdomain'].str.lower().str.count(r'[aeiou]')
        # Cuenta consonantes
        data_urls['dom_cons'] = data_urls['domain_subdomain'].str.lower().str.count(r'[a-z]') - data_urls['dom_vocales']
        # Cuenta números
        data_urls['dom_num'] = data_urls['domain_subdomain'].str.count('\d')
        # Cuenta cantidad de caracteres diferentes
        data_urls['dom_car_dif'] = data_urls['domain_subdomain'].apply(set).apply(len)

        def dummy_ip(columna):
            if columna == '':
                return 1
            else:
                return 0

        # Dominio es IP
        data_urls['dom_ip'] = data_urls['suffix'].apply(dummy_ip)

        #ip_dummies = pd.get_dummies(data_urls['suffix'] == '')
        #if ip_dummies.shape[1] == 2:
        #    data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '', drop_first=True)
        #elif ip_dummies.shape[1] == 1:
        #    if ip_dummies.columns[0] == False:
        #        data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '').replace(1,0)
        #    else:
        #        data_urls['dom_ip'] = pd.get_dummies(data_urls['suffix'] == '')
            
        # Variable dummy Scheme
        sch_dummies = pd.get_dummies(data_urls['scheme'], prefix='sch')
        data_urls = pd.concat([data_urls, sch_dummies], axis = 1)

        # Variables de Suffix
        data_urls['suf_len'] = data_urls['suffix'].str.len()

        # Creamos e imprimimos una lista con el top 5 de sufijos.
        top_suf_list = ['com', 'net', 'org', 'ru', 'xyz']
        
        def dummy_com(columna):
            if columna == 'com':
                return 1
            else:
                return 0        
        def dummy_net(columna):
            if columna == 'com':
                return 1
            else:
                return 0
        def dummy_org(columna):
            if columna == 'com':
                return 1
            else:
                return 0
        def dummy_ru(columna):
            if columna == 'com':
                return 1
            else:
                return 0
        def dummy_xyz(columna):
            if columna == 'com':
                return 1
            else:
                return 0
        def dummy_other(columna):
            if columna == 'other':
                return 1
            else:
                return 0

        data_urls['suffix2'] = data_urls['suffix']
        # Asignamos categoría 'other' a todas las clases que no pertenezcan a top_suf_list.
        data_urls.loc[data_urls['suffix2'].isin(top_suf_list).apply(np.bitwise_not), 'suffix2'] = 'other'
        # Armamos las columnas dummy.
        data_urls['suf_com'] = data_urls['suffix2'].apply(dummy_com)
        data_urls['suf_net'] = data_urls['suffix2'].apply(dummy_net)
        data_urls['suf_org'] = data_urls['suffix2'].apply(dummy_org)
        data_urls['suf_ru'] = data_urls['suffix2'].apply(dummy_ru)
        data_urls['suf_xyz'] = data_urls['suffix2'].apply(dummy_xyz)
        data_urls['suf_other'] = data_urls['suffix2'].apply(dummy_other)
        
        #suf_dummies = pd.get_dummies(data_urls['suffix2'], prefix='suf')
        #data_urls = pd.concat([data_urls, suf_dummies], axis = 1)

        for domain in metric_domains:
            data_urls['metric_ds_'+domain] = data_urls['domain_subdomain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_d_'+domain] = data_urls['domain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_s_'+domain] = data_urls['subdomain'].apply(lambda x: jellyfish.jaro_winkler(x, domain))
            data_urls['metric_p_'+domain] = data_urls['path'].apply(lambda x: jellyfish.jaro_winkler(x, domain))

In [23]:
data_urls

,url,scheme,domain_complete,domain,subdomain,suffix,domain_subdomain,path,dom_n_puntos,dom_n_guion,...,metric_s_rakuten,metric_p_rakuten,metric_ds_steam,metric_d_steam,metric_s_steam,metric_p_steam,metric_ds_olx,metric_d_olx,metric_s_olx,metric_p_olx
0,https://git-scm.com/docs/gitignore,https,git-scm.com,git-scm,,com,git-scm,/docs/gitignore,0,1,...,0.000000,0.473016,0.561905,0.561905,0.000000,0.422222,0.000000,0.000000,0.0,0.466667
1,https://scikit-learn.org/stable/modules/model_...,https,scikit-learn.org,scikit-learn,,org,scikit-learn,/stable/modules/model_persistence.html,0,1,...,0.000000,0.392231,0.522222,0.522222,0.000000,0.643860,0.000000,0.000000,0.0,0.406433
2,https://www.lanacion.com.ar/,https,lanacion.com.ar,lanacion,,com.ar,lanacion,/,0,0,...,0.000000,0.000000,0.441667,0.441667,0.000000,0.000000,0.486111,0.486111,0.0,0.000000
3,https://articulo.mercadolibre.com.ar/MLA-73710...,https,articulo.mercadolibre.com.ar,mercadolibre,articulo.,com.ar,articulo.mercadolibre,/MLA-737102690-sommier-king-la-cardeuse-native...,1,0,...,0.505291,0.464977,0.496825,0.505556,0.374074,0.538172,0.420635,0.000000,0.0,0.566308
4,https://www.clarin.com/,https,clarin.com,clarin,,com,clarin,/,0,0,...,0.000000,0.000000,0.455556,0.455556,0.000000,0.000000,0.500000,0.500000,0.0,0.000000


In [5]:
detector.prepararInput(prueba)

In [9]:
detector.data.iloc[:,10:30]

,dom_car_dif,dom_ip,sch_https,suf_len,suf_com,suf_net,suf_org,suf_ru,suf_xyz,suf_other,metric_ds_amazon,metric_d_amazon,metric_s_amazon,metric_p_amazon,metric_ds_instagram,metric_d_instagram,metric_s_instagram,metric_p_instagram,metric_ds_google,metric_d_google
0,7,0,1,3,1,1,1,1,1,0,0.000000,0.000000,0.000000,0.488889,0.588624,0.588624,0.000000,0.400000,0.436508,0.436508
1,11,0,1,3,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.526316,0.590741,0.590741,0.000000,0.495712,0.500000,0.500000
2,6,0,1,6,0,0,0,0,0,1,0.638889,0.638889,0.000000,0.000000,0.324074,0.324074,0.000000,0.000000,0.000000,0.000000
3,13,0,1,6,0,0,0,0,0,1,0.436508,0.472222,0.518519,0.504659,0.484127,0.296296,0.314815,0.547640,0.436508,0.472222
4,6,0,1,3,1,1,1,1,1,0,0.555556,0.555556,0.000000,0.000000,0.518519,0.518519,0.000000,0.000000,0.000000,0.000000


In [16]:
detector.data.iloc[:,9:30]

,dom_num,dom_car_dif,dom_ip,sch_https,suf_len,suf_com,suf_org,suf_other,metric_ds_amazon,metric_d_amazon,...,metric_p_amazon,metric_ds_instagram,metric_d_instagram,metric_s_instagram,metric_p_instagram,metric_ds_google,metric_d_google,metric_s_google,metric_p_google,metric_ds_whatsapp
0,0,7,0,1,3,1,0,0,0.000000,0.000000,...,0.488889,0.588624,0.588624,0.000000,0.400000,0.436508,0.436508,0.000000,0.322222,0.511905
1,0,11,0,1,3,0,1,0,0.000000,0.000000,...,0.526316,0.590741,0.590741,0.000000,0.495712,0.500000,0.500000,0.000000,0.423977,0.430556
2,0,6,0,1,6,0,0,1,0.638889,0.638889,...,0.000000,0.324074,0.324074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
3,0,13,0,1,6,0,0,1,0.436508,0.472222,...,0.504659,0.484127,0.296296,0.314815,0.547640,0.436508,0.472222,0.425926,0.410394,0.505952
4,0,6,0,1,3,1,0,0,0.555556,0.555556,...,0.000000,0.518519,0.518519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.430556


In [6]:
urls = ['http://git-scm.com/docs/gitignore', 
        'http://scikit-learn.org/stable/modules/model_persistence.html',
        'http://www.lanacion.com.ar/',
        'http://articulo.mercadolibre.com.ar/MLA-737102690-sommier-king-la-cardeuse-native-400-200x200-_JM?variation=95000501490#reco_item_pos=2&reco_backend=promotions-sorted-by-score-mla-A&reco_backend_type=low_level&reco_client=seller-promotions&reco_id=574ab7bd-21ab-43a8-a868-69b5785b0fc9&deal_print_id=25c97840-04f2-11ec-85d9-336a60e0da68&c_id=carouseldynamic-home&c_element_order=undefined&c_campaign=OFERTAS-X-24-HORAS&c_uid=25c97840-04f2-11ec-85d9-336a60e0da68',
        'http://www.clarin.com/', 
        'http://amoueaom-cc-jp.wnejpkv.cn/',
        'http://abdcenter.rhinosme-stagging.com/.kkklll', 
        ' http://foundations-admin-10000000003216566325623260.ml/checkpoint_next.php ',
        'http://pagefbscmaintenenssupporss.ga/', 
        'http://bit.ly/3sFQZza?/identityconfirmation',
        'http://yahoomailerpotr.weebly.com/', 
        'http://freenltro.ru', 
        'http://www.brookssiparis.com/login.html', 
        'https://minute-too-late.my.id/',
        'https://guitar.3utilities.com/', 
        'https://dealsmart247.com/', 
        'https://www.amazon.de/gp/r.html', 
        'https://www.hsbc.com.hk/', 
        'https://www.eurodns.com/', 
        'https://openprovider.com/', 
        'https://www.cloudflare.com/abuse/', 
        'https://www.lanacion.com.ar/politica/la-estrategia-de-alberto-fernandez-para-cerrar-el-caso-del-cumpleanos-en-15-dias-y-sin-pisar-nid24082021/',
        'https://www.odoo.com/', 'http://www.jobfinder.com.vn/jobs?ts=ya&q=dhl%20express%20job', 
        'https://apple.com.pa/', 'https://apple.com.pe/', 'http://www.t-mobile.com']

prueba = pd.DataFrame({'url':urls})

**Hay que corregir la generación de dummies, get_dummies genera con las que están presentes pero no considera las que no aparecen. Hay que hacerlas fijas armando funciones para cada una.**

In [7]:
detector.calcularProbabilidad(prueba)

array([[0.77, 0.23],
       [0.86, 0.14],
       [0.25, 0.75],
       [0.71, 0.29],
       [0.96, 0.04],
       [0.31, 0.69],
       [0.47, 0.53],
       [0.5 , 0.5 ],
       [0.47, 0.53],
       [0.56, 0.44],
       [0.29, 0.71],
       [0.9 , 0.1 ],
       [0.63, 0.37],
       [0.06, 0.94],
       [0.17, 0.83],
       [0.44, 0.56],
       [0.46, 0.54],
       [0.38, 0.62],
       [0.6 , 0.4 ],
       [0.45, 0.55],
       [0.49, 0.51],
       [0.92, 0.08],
       [0.55, 0.45],
       [0.36, 0.64],
       [0.36, 0.64],
       [0.96, 0.04]])

In [50]:
prueba.iloc[:,30:50]

,metric_ds_amazon,metric_d_amazon,metric_s_amazon,metric_p_amazon,metric_ds_instagram,metric_d_instagram,metric_s_instagram,metric_p_instagram,metric_ds_google,metric_d_google,metric_s_google,metric_p_google,metric_ds_whatsapp,metric_d_whatsapp,metric_s_whatsapp,metric_p_whatsapp,metric_ds_twitter,metric_d_twitter,metric_s_twitter,metric_p_twitter
0,0.000000,0.000000,0.000000,0.488889,0.588624,0.588624,0.000000,0.400000,0.436508,0.436508,0.000000,0.322222,0.511905,0.511905,0.000000,0.294444,0.357143,0.357143,0.000000,0.473016
1,0.000000,0.000000,0.000000,0.526316,0.590741,0.590741,0.000000,0.495712,0.500000,0.500000,0.000000,0.423977,0.430556,0.430556,0.000000,0.451754,0.551587,0.551587,0.000000,0.502506
2,0.638889,0.638889,0.000000,0.000000,0.324074,0.324074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.422619,0.422619,0.000000,0.000000
3,0.436508,0.472222,0.518519,0.504659,0.484127,0.296296,0.314815,0.547640,0.436508,0.472222,0.425926,0.410394,0.505952,0.402778,0.490741,0.363351,0.420635,0.484127,0.502646,0.492320
4,0.555556,0.555556,0.000000,0.000000,0.518519,0.518519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.430556,0.430556,0.000000,0.000000,0.436508,0.436508,0.000000,0.000000
5,0.532828,0.000000,0.561111,0.000000,0.378788,0.417989,0.511111,0.000000,0.434343,0.000000,0.455556,0.000000,0.503788,0.511905,0.525000,0.000000,0.396104,0.428571,0.403175,0.000000
6,0.425926,0.447712,0.511111,0.000000,0.574074,0.619826,0.531481,0.000000,0.302469,0.483660,0.422222,0.430556,0.466049,0.495098,0.483333,0.000000,0.406526,0.518908,0.465079,0.000000
7,0.000000,0.000000,0.000000,0.499423,0.000000,0.000000,0.000000,0.527658,0.000000,0.000000,0.000000,0.453102,0.000000,0.000000,0.000000,0.553752,0.000000,0.000000,0.000000,0.457792
8,0.606838,0.606838,0.000000,0.000000,0.607194,0.607194,0.000000,0.000000,0.470085,0.470085,0.000000,0.000000,0.539103,0.539103,0.000000,0.000000,0.403541,0.403541,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.430556,0.629630,0.629630,0.000000,0.490741,0.000000,0.000000,0.000000,0.000000,0.486111,0.486111,0.000000,0.500000,0.484127,0.484127,0.000000,0.000000


In [17]:
detector.data

,dom_n_puntos,dom_n_guion,dom_n_guionbajo,dom_len_tot,dom_len,dom_len_sub,url_len,dom_vocales,dom_cons,dom_num,...,metric_s_rakuten,metric_p_rakuten,metric_ds_steam,metric_d_steam,metric_s_steam,metric_p_steam,metric_ds_olx,metric_d_olx,metric_s_olx,metric_p_olx
0,0,1,0,7,7,0,34,1,5,0,...,0.000000,0.473016,0.561905,0.561905,0.000000,0.422222,0.000000,0.000000,0.000000,0.466667
1,0,1,0,12,12,0,62,4,7,0,...,0.000000,0.392231,0.522222,0.522222,0.000000,0.643860,0.000000,0.000000,0.000000,0.406433
2,0,0,0,8,8,0,28,4,4,0,...,0.000000,0.000000,0.441667,0.441667,0.000000,0.000000,0.486111,0.486111,0.000000,0.000000
3,1,0,0,21,12,9,461,9,11,0,...,0.505291,0.464977,0.496825,0.505556,0.374074,0.538172,0.420635,0.000000,0.000000,0.566308
4,0,0,0,6,6,0,23,2,4,0,...,0.000000,0.000000,0.455556,0.455556,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000
5,1,2,0,22,7,15,34,7,12,0,...,0.542857,0.000000,0.467677,0.447619,0.488889,0.000000,0.459596,0.000000,0.466667,0.000000
6,1,1,0,27,17,10,46,8,17,0,...,0.573810,0.422619,0.566049,0.481046,0.366667,0.000000,0.000000,0.464052,0.000000,0.000000
7,0,0,0,0,0,0,77,0,0,0,...,0.000000,0.457792,0.000000,0.000000,0.000000,0.533983,0.000000,0.000000,0.000000,0.448773
8,0,0,0,26,26,0,38,8,18,0,...,0.000000,0.000000,0.597436,0.597436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0,0,0,3,3,0,44,1,2,0,...,0.000000,0.000000,0.511111,0.511111,0.000000,0.441667,0.000000,0.000000,0.000000,0.000000
